In [1]:
import time
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as ss
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, plot_confusion_matrix, plot_roc_curve, classification_report
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from matplotlib import colors
import warnings
import os
import numpy as np
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn.model_selection import train_test_split
from math import sin, cos, sqrt, atan2, radians
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

import tensorflow as tf
from tensorflow import keras
import os
import time
from datetime import timedelta
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats as ss
import matplotlib.colors
#from yellowbrick.classifier import ClassPredictionError

from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, plot_confusion_matrix, plot_roc_curve, classification_report
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression, SGDClassifier
#from tqdm import tqdm_notebook 
#import imageio

import statistics
from statistics import mode

#importing plotly and cufflinks in offline mode
#import cufflinks as cf
#import plotly.offline
#cf.go_offline()
#cf.set_config_file(offline=False, world_readable=True)
#import plotly.express as px

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import tensorflow as tf
from tensorflow import keras


In [2]:
# Files 

attacks = pd.read_csv("Attack.csv", low_memory = False)
env = pd.read_csv("environmentMonitoring.csv", low_memory = False)
pat = pd.read_csv("patientMonitoring.csv", low_memory = False)

df = pd.concat([attacks, env, pat]).reset_index(drop=True)

In [3]:
df.drop('class',axis=1,inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188694 entries, 0 to 188693
Data columns (total 51 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   frame.time_delta       188694 non-null  float64
 1   frame.time_relative    188694 non-null  float64
 2   frame.len              188694 non-null  int64  
 3   ip.src                 188694 non-null  object 
 4   ip.dst                 188694 non-null  object 
 5   tcp.srcport            188694 non-null  int64  
 6   tcp.dstport            188694 non-null  int64  
 7   tcp.flags              188694 non-null  object 
 8   tcp.time_delta         188694 non-null  float64
 9   tcp.len                188694 non-null  int64  
 10  tcp.ack                188694 non-null  int64  
 11  tcp.connection.fin     188694 non-null  float64
 12  tcp.connection.rst     188694 non-null  float64
 13  tcp.connection.sack    188694 non-null  float64
 14  tcp.connection.syn     188694 non-nu

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
frame.time_delta,188694.0,0.077846,0.400776,0.0,0.000012,0.000042,0.000143,4.443631e+01
frame.time_relative,188694.0,1374.247508,1806.634733,0.0,37.687817,431.028124,2075.031445,6.611038e+03
frame.len,188694.0,164.856943,329.238186,54.0,70.000000,78.000000,85.000000,1.766000e+03
tcp.srcport,188694.0,35293.644022,19335.803261,1883.0,33387.000000,39979.000000,51684.000000,6.547100e+04
tcp.dstport,188694.0,12652.982241,20280.180294,1883.0,1883.000000,1883.000000,1883.000000,6.547100e+04
tcp.time_delta,188694.0,1.870432,6.482882,0.0,0.000033,0.504395,2.000981,6.003733e+01
tcp.len,188694.0,101.774004,331.691920,0.0,4.000000,10.000000,18.000000,1.698000e+03
tcp.ack,188694.0,66682.917883,226307.573429,0.0,49.000000,175.000000,9029.000000,1.712906e+06
tcp.connection.fin,188694.0,0.008750,0.093130,0.0,0.000000,0.000000,0.000000,1.000000e+00
tcp.connection.rst,188694.0,0.008654,0.092625,0.0,0.000000,0.000000,0.000000,1.000000e+00


In [6]:
df.describe(include=object).T

,count,unique,top,freq
ip.src,188694,44,192.168.1.90,46210
ip.dst,188694,44,10.5.126.56,72266
tcp.flags,188694,8,0x00000018,148212
tcp.payload,188694,50526,0,30063
tcp.checksum,188694,45930,0x0000110f,10750
mqtt.clientid,188694,1158,0,187377
mqtt.conack.flags,188694,2,0,186721
mqtt.conflags,188694,2,0,186721
mqtt.hdrflags,188694,14,0x00000030,82517
mqtt.msg,188694,24714,0,44451


In [7]:
df['tcp.checksum']

0         0x00009a5d
1         0x00009941
2         0x0000bb2e
3         0x00000e6e
4         0x0000f752
             ...    
188689    0x00001118
188690    0x00001105
188691    0x00001105
188692    0x00001119
188693    0x00001119
Name: tcp.checksum, Length: 188694, dtype: object

In [8]:
pd.DataFrame([df.nunique(), df.dtypes]).T

,0,1
frame.time_delta,11904,float64
frame.time_relative,176587,float64
frame.len,633,int64
ip.src,44,object
ip.dst,44,object
tcp.srcport,2062,int64
tcp.dstport,2062,int64
tcp.flags,8,object
tcp.time_delta,49228,float64
tcp.len,638,int64


In [9]:
single_val_cols=[]
for col in df.columns:
    if df[col].nunique()==1:
        single_val_cols.append(col)
    else:
        pass
df = df.drop(single_val_cols, axis=1)


In [10]:
cat_cols = []
for col in df.columns:
    if df[col].nunique()<100:
        cat_cols.append(col)
df[cat_cols] = df[cat_cols].astype(object)

df['tcp.flags'] = df['tcp.flags'].apply(lambda x: x[-3:-1])
df['mqtt.msg'] = df['mqtt.msg'].apply(lambda x: len(str(x)))
df['tcp.flags'] = df['tcp.flags'].astype('float64')
df['mqtt.msg'] = df['mqtt.msg'].astype('float64')
df['tcp.flags'] = df['tcp.flags'].astype(object)
df['label'] = df['label'].astype('float64')

In [31]:
num_df = df.select_dtypes(['int64','float64','uint8'])
num_df.shape

(188694, 113)

In [32]:
cat_df = df.select_dtypes(object)
cat_df.shape

(188694, 7)

In [13]:
num_df.nunique()


frame.time_delta        11904
frame.time_relative    176587
frame.len                 633
tcp.srcport              2062
tcp.dstport              2062
tcp.time_delta          49228
tcp.len                   638
tcp.ack                 26555
mqtt.msg                   48
label                       2
dtype: int64

In [14]:
cat_df.nunique()

ip.src                      44
ip.dst                      44
tcp.flags                    2
tcp.connection.fin           2
tcp.connection.rst           2
tcp.connection.sack          2
tcp.connection.syn           2
tcp.flags.ack                2
tcp.flags.fin                2
tcp.flags.push               2
tcp.flags.reset              2
tcp.flags.syn                2
tcp.hdr_len                  4
tcp.payload              50526
tcp.pdu.size                84
tcp.window_size_value       57
tcp.checksum             45930
mqtt.clientid             1158
mqtt.clientid_len            9
mqtt.conack.flags            2
mqtt.conflags                2
mqtt.dupflag                 2
mqtt.hdrflags               14
mqtt.kalive                  7
mqtt.len                    83
mqtt.msgtype                11
mqtt.qos                     2
mqtt.retain                  2
mqtt.topic               23440
mqtt.topic_len              37
mqtt.ver                     2
ip.ttl                       2
dtype: i

In [15]:
df['payload_len'] = df['tcp.payload'].apply(lambda x: len(str(x).split(sep=':')))
df['payload_str'] =  df['tcp.payload'].apply(lambda x: (str(x).split(sep=':'))[0])
df['payload_end'] =  df['tcp.payload'].apply(lambda x: (str(x).split(sep=':'))[-1])

In [16]:
pay_freq = df['tcp.payload'].value_counts().to_dict()
df['payload_freq'] = df['tcp.payload'].map(pay_freq)

In [17]:
df.drop('tcp.payload',axis=1,inplace=True)
df['payload_len'] = df['payload_len'].astype('float64')
df['payload_freq'] = df['payload_freq'].astype('float64')

In [18]:
OH_cols = []
for col in df.drop('label',axis=1).columns:
    if df[col].nunique()<5:
        OH_cols.append(col)

In [19]:
df = pd.get_dummies(df,columns=OH_cols)

In [20]:
check_freq = df['tcp.checksum'].value_counts().to_dict()
df['checksum_freq'] = df['tcp.checksum'].map(check_freq)
df['checksum_freq'] = df['checksum_freq'].astype('float64')
df.drop('tcp.checksum',axis=1,inplace=True)

In [21]:
hdrflags_freq = df['mqtt.hdrflags'].value_counts().to_dict()
df['hdrflags_freq'] = df['mqtt.hdrflags'].map(hdrflags_freq)
df['hdrflags_freq'] = df['hdrflags_freq'].astype('float64')

In [22]:
df["mqtt.hdrflags"] = df["mqtt.hdrflags"].apply(lambda x: x.replace(x,'0') if x in ['0x00000031','0x000000e0','0x00000050','0x00000090','0x00000082'] else x)

In [23]:
Oh_cols2 = ['mqtt.kalive', 'mqtt.hdrflags', 'mqtt.clientid_len', 'mqtt.msgtype']
df = pd.get_dummies(df,columns=Oh_cols2)

In [24]:
topics = list(df['mqtt.topic'].value_counts()[1:17].index)
for topic in topics:
    df[topic] = df['mqtt.topic'].apply(lambda x: 1 if x == topic else 0)
df.drop('mqtt.topic',axis=1,inplace=True)

In [25]:
top_src_ip = df['ip.src'].value_counts()[:6].index
for ip in top_src_ip:
    df['src_ip'+ip] = df['ip.src'].apply(lambda x: 1 if x == topic else 0)
df.drop('ip.src',axis=1,inplace=True)

In [26]:
top_dst_ip = df['ip.dst'].value_counts()[:3].index
for ip in top_dst_ip:
    df['dst_ip '+ip] = df['ip.dst'].apply(lambda x: 1 if x == topic else 0)
df.drop('ip.dst',axis=1,inplace=True)

In [33]:
df2 = num_df

In [54]:
X,y = df2.drop('label',axis=1), df2['label']

In [55]:
X_train_val,X_test, y_train_val , y_test = train_test_split(X,y, random_state = 0, test_size = 0.15,shuffle=True)

In [56]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val,y_train_val, random_state = 0, test_size = 0.15, shuffle=True)

In [57]:
def tf_preprocess(x, y):
    x = tf.cast(x, tf.float32)
    y = tf.cast(y, tf.int64)
    return x, y


def tf_create_dataset(xs, ys, n_classes=2):
    ys = tf.one_hot(ys, depth=n_classes)
    return tf.data.Dataset.from_tensor_slices((xs, ys)) \
      .map(tf_preprocess) \
      .shuffle(len(ys)) \
      .batch(128)


def evaluate_NN(networks,x_test, y_test):

    class_wise_accuracy = {}
    metrics = {}

    # Metrics
    for network in range(len(networks)):
        pred = networks[network].predict(x_test)
        pred = np.argmax(pred, axis=1)
        accuracy = accuracy_score(y_test, pred)

        f1_macro = f1_score(y_test,pred,average="macro")
        f1_micro = f1_score(y_test,pred,average="micro")
        f1_weighted = f1_score(y_test,pred,average="weighted")

        precision_macro = precision_score(y_test, pred, average='macro')
        precision_micro = precision_score(y_test, pred, average='micro')
        precision_weighted = precision_score(y_test, pred, average='weighted')

        recall_macro = recall_score(y_test, pred, average='macro')
        recall_micro = recall_score(y_test, pred, average='micro')
        recall_weighted = recall_score(y_test, pred, average='weighted')

        metrics.update({'NN_'+str(network+1):[accuracy,f1_macro,f1_micro,f1_weighted,precision_macro,precision_micro,precision_weighted,recall_macro,recall_micro,recall_weighted]})

        # Class Wise Accuracy
        a = pd.crosstab(y_test,pred)
        accuracy_matrix = (a.max(axis=1)/a.sum(axis=1))
        class_wise_accuracy.update({'NN_'+str(network+1):accuracy_matrix})

    # Combining Metrics and Classwise accuracy
    
    metric_df = pd.DataFrame(metrics, index=["Accuracy","F1_Macro","F1_Micro","F1_Weighted",
                                            "Precision_Macro","Precision_Micro","Precision_Weighted",
                                            "Recall_Macro","Recall_Micro", "Recall_Weighted"]).T

    class_accuracy_df = pd.DataFrame(class_wise_accuracy).T

    final_df = metric_df.join(class_accuracy_df)

    return final_df


def build_NN(hid_layers_count, neuron_per_layer, hid_activation, out_neurons_count):

    models_layers = []

    for layer in range(hid_layers_count):
        layer = keras.layers.Dense(units=neuron_per_layer[layer], activation=hid_activation, kernel_initializer=tf.keras.initializers.HeNormal())
        models_layers.append(layer)

    out_layer = keras.layers.Dense(units= out_neurons_count, activation= 'softmax', kernel_initializer=tf.keras.initializers.HeNormal())
    models_layers.append(out_layer)

    model= keras.Sequential(models_layers)

    return model


def fit_NN(model, x_train, y_train, x_val, y_val, epochs, steps_per_epoch, opt, loss_fn, metrics):


    train = tf_create_dataset(x_train, y_train)
    val = tf_create_dataset(x_val, y_val)

    model.compile(optimizer=opt, loss=loss_fn, metrics=metrics)

    model.fit(
        train.repeat(), 
        epochs=epochs, 
        steps_per_epoch=steps_per_epoch,
        validation_data=val, 
        )

    return model


def NN_tuner(train,y,hlayers,neurons_count,optimizers,epochs_list,steps_per_epoch,h_acti,o_acti,out_neurons_count,loss_fn,metrics):

    all_models={}
    for hidden_layer in range(hlayers):
        model_layers = []
        for opt in optimizers:
            for epoch in epochs_list:
                for steps in steps_per_epoch:
                    for hid_act in h_acti:
                        for out_act in o_acti:
                            for neuron_per_layer in neurons_count:
                                layer = keras.layers.Dense(units=neuron_per_layer, activation=hid_act)
                                model_layers.append(layer)
                            
                            out_layer = keras.layers.Dense(units= out_neurons_count, activation= out_act)
                            model_layers.append(out_layer)
                            model = keras.Sequential(model_layers)

                            model.compile(optimizer=opt, loss=loss_fn, metrics=metrics)
                            


                            model.fit(
                                train.repeat(), 
                                epochs=epoch, 
                                steps_per_epoch=steps,
                                validation_data=test, 
                                )
                            
                            all_models.update({
                                model:
                                    [{'hidden_layer' : hidden_layer,
                                    'Neurons per layer' : neuron_per_layer,
                                    'optimizer' : opt,
                                    'Epochs':epoch,
                                    'Steps_per_epoch' : steps,
                                    'Hidden Layer activation' : hid_act,
                                    'Output Layer Activation' : out_act}]
                                })
                            

    return all_models

def NN_bagging_predict(models,X):
    bag_pred =[]
    for model in models:
        pred = model.predict(X)
        pred = np.argmax(pred, axis=1)
        bag_pred.append(pred)
        
    final_pred = scipy.stats.mode(np.stack(bag_pred), axis=0)

    return final_pred.mode[0]

In [58]:
# Scaling the data
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(X_train)
x_val_scaled = scaler.transform(X_val)
x_test_scaled = scaler.transform(X_test)

In [66]:
# FNN on Random Under Sampled Data
basic_model = build_NN(1, [1], tf.keras.layers.LeakyReLU(alpha=0.01),2)
basic_model = fit_NN(basic_model, x_train_scaled, y_train, x_val_scaled, y_val,10, 128, 'adam',  tf.losses.CategoricalCrossentropy(from_logits=True), ['accuracy'])

Epoch 1/10
128/128 [==============================] - 1s 5ms/step - loss: 0.6903 - accuracy: 0.5601 - val_loss: 0.6871 - val_accuracy: 0.5734
Epoch 2/10
128/128 [==============================] - 0s 4ms/step - loss: 0.6806 - accuracy: 0.5786 - val_loss: 0.6231 - val_accuracy: 0.5734
Epoch 3/10
128/128 [==============================] - 1s 4ms/step - loss: 0.5466 - accuracy: 0.6068 - val_loss: 0.4987 - val_accuracy: 0.9576
Epoch 4/10
128/128 [==============================] - 0s 4ms/step - loss: 0.4742 - accuracy: 0.9753 - val_loss: 0.4508 - val_accuracy: 0.9860
Epoch 5/10
128/128 [==============================] - 0s 4ms/step - loss: 0.4327 - accuracy: 0.9872 - val_loss: 0.4197 - val_accuracy: 0.9927
Epoch 6/10
128/128 [==============================] - 0s 4ms/step - loss: 0.4089 - accuracy: 0.9930 - val_loss: 0.3976 - val_accuracy: 0.9930
Epoch 7/10
128/128 [==============================] - 0s 4ms/step - loss: 0.3886 - accuracy: 0.9945 - val_loss: 0.3820 - val_accuracy: 0.9986
Epoch 

In [67]:
evaluate_NN([basic_model],x_test_scaled,y_test)

,Accuracy,F1_Macro,F1_Micro,F1_Weighted,Precision_Macro,Precision_Micro,Precision_Weighted,Recall_Macro,Recall_Micro,Recall_Weighted,0.0,1.0
NN_1,0.998869,0.998841,0.998869,0.99887,0.998784,0.998869,0.99887,0.998898,0.998869,0.998869,0.998718,0.999077


In [124]:
def classwise_accuracy(true, pred):
   a = pd.crosstab(true,pred)
   return (a.max(axis=1)/a.sum(axis=1))


def build_models(X,y):
    models={}
    training_times = {}
    
    # Logistic Regression    
    lr = LogisticRegression()
    start_time = time.time()
    lr.fit(X,y)
    elapsed_time_secs = timedelta(seconds=round(time.time() - start_time))
    models.update({"Logistic Regression":lr})
    training_times.update({"Logistic Regression":elapsed_time_secs.seconds})

    # KNN
    knn = KNeighborsClassifier(n_neighbors=3)
    start_time = time.time()
    knn.fit(X,y)
    elapsed_time_secs = timedelta(seconds=round(time.time() - start_time))
    models.update({"KNN":knn})
    training_times.update({"KNN":elapsed_time_secs.seconds})
    
    # Decision Tree
    dt = DecisionTreeClassifier()
    start_time = time.time()
    dt.fit(X,y)
    elapsed_time_secs = timedelta(seconds=round(time.time() - start_time))
    models.update({"Decision Tree":dt})
    training_times.update({"Decision Tree":elapsed_time_secs.seconds})
    
    # XGBoost
    xg = XGBClassifier()
    start_time = time.time()
    xg.fit(X,y)
    elapsed_time_secs = timedelta(seconds=round(time.time() - start_time))
    models.update({"XG Boost":xg})
    training_times.update({"XG Boost":elapsed_time_secs.seconds})
    
    
    # XGBoost Random Forrest
    xgrf = XGBRFClassifier()
    start_time = time.time()
    xgrf.fit(X,y)
    elapsed_time_secs = timedelta(seconds=round(time.time() - start_time))
    models.update({"XG Random Forest":xgrf})
    training_times.update({"XG Random Forest":elapsed_time_secs.seconds})
    
    # Random Forest
    rf = RandomForestClassifier()
    start_time = time.time()
    rf.fit(X,y)
    elapsed_time_secs = timedelta(seconds=round(time.time() - start_time))
    models.update({"Random Forest":rf })
    training_times.update({"Random Forest":elapsed_time_secs.seconds})
    
    training_dur = pd.DataFrame(training_times, index=["Training Time"]).T
    return models, training_dur


def model_evaluation(models, X_test, y_test,training_df):
    scores={}  
    class_wise_accuracy = {}

    for model in models:
        pred = models[model].predict(X_test)
        accuracy = accuracy_score(y_test, pred)

        f1_macro = f1_score(y_test,pred,average="macro")
        f1_micro = f1_score(y_test,pred,average="micro")
        f1_weighted = f1_score(y_test,pred,average="weighted")

        precision_macro = precision_score(y_test, pred, average='macro')
        precision_micro = precision_score(y_test, pred, average='micro')
        precision_weighted = precision_score(y_test, pred, average='weighted')

        recall_macro = recall_score(y_test, pred, average='macro')
        recall_micro = recall_score(y_test, pred, average='micro')
        recall_weighted = recall_score(y_test, pred, average='weighted')

        #roc_auc = roc_auc_score(y_test, pred, , average='macro')

        scores.update({model:[accuracy,f1_macro,f1_micro,f1_weighted,precision_macro,precision_micro,precision_weighted,recall_macro,recall_micro,recall_weighted]})

        #pred = models[model].predict(X_test)
        a = pd.crosstab(y_test,pred)
        accuracy_matrix = (a.max(axis=1)/a.sum(axis=1))
        class_wise_accuracy.update({model:accuracy_matrix})



    metric_df = pd.DataFrame(scores, index=["Accuracy","F1_Macro","F1_Micro","F1_Weighted",
                                            "Precision_Macro","Precision_Micro","Precision_Weighted",
                                            "Recall_Macro","Recall_Micro", "Recall_Weighted"]).T

    class_accuracy_df = pd.DataFrame(class_wise_accuracy).T
    metric_and_training_dur = pd.merge(metric_df,training_df, left_index=True, right_index=True)

    final_df = metric_and_training_dur.join(class_accuracy_df)

    return final_df

In [112]:
basic_models = build_models(x_train_scaled,y_train)

E:\Anaconda\envs\healthapp\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:31:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:31:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [125]:
model_evaluation(basic_models[0],x_test_scaled,y_test,basic_models[1])

,Accuracy,F1_Macro,F1_Micro,F1_Weighted,Precision_Macro,Precision_Micro,Precision_Weighted,Recall_Macro,Recall_Micro,Recall_Weighted,Training Time,0.0,1.0
Logistic Regression,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2,1.0,1.0
KNN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,1.0,1.0
Decision Tree,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,1.0,1.0
XG Boost,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,1.0,1.0
XG Random Forest,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,1.0,1.0
Random Forest,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,7,1.0,1.0


In [120]:
for i in basic_models[0]:
    print(i)

Logistic Regression
KNN
Decision Tree
XG Boost
XG Random Forest
Random Forest


In [50]:
X

,frame.time_delta,frame.time_relative,frame.len,tcp.srcport,tcp.dstport,tcp.time_delta,tcp.len,tcp.ack,mqtt.msg,payload_len,...,InfusionPump,src_ip192.168.1.90,src_ip192.168.1.91,src_ip10.5.126.145,src_ip10.5.126.147,src_ip10.5.126.148,src_ip10.5.126.146,dst_ip 10.5.126.56,dst_ip 192.168.1.91,dst_ip 192.168.1.90
0,0.000000,0.000000,74,56808,1883,0.000000,0,0,3.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,0.000052,0.000052,74,1883,56808,0.000052,0,1,3.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,0.000008,0.000060,74,56810,1883,0.000000,0,0,3.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,0.000012,0.000072,74,1883,56810,0.000012,0,1,3.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,0.000003,0.000075,74,56812,1883,0.000000,0,0,3.0,1.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188689,0.000196,6611.037621,78,44993,1883,1.989358,10,225,5.0,10.0,...,0,0,0,0,0,0,0,0,0,0
188690,0.000172,6611.037793,77,38323,1883,1.989338,9,225,4.0,9.0,...,0,0,0,0,0,0,0,0,0,0
188691,0.000126,6611.037919,77,41889,1883,1.989337,9,225,4.0,9.0,...,0,0,0,0,0,0,0,0,0,0
188692,0.000146,6611.038065,77,35965,1883,1.989211,9,225,4.0,9.0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
X_test['label_true'] = y_test

In [84]:
X_test

,frame.time_delta,frame.time_relative,frame.len,tcp.srcport,tcp.dstport,tcp.time_delta,tcp.len,tcp.ack,mqtt.msg,payload_len,...,src_ip192.168.1.91,src_ip10.5.126.145,src_ip10.5.126.147,src_ip10.5.126.148,src_ip10.5.126.146,dst_ip 10.5.126.56,dst_ip 192.168.1.91,dst_ip 192.168.1.90,label_true,pred
132052,0.000042,367.029651,86,33571,1883,1.999646,18,17,4.0,18.0,...,0,0,0,0,0,0,0,0,0.0,0
147977,0.000112,2113.041946,86,37701,1883,2.009467,18,75,4.0,18.0,...,0,0,0,0,0,0,0,0,0.0,0
58671,0.000165,53.493371,62,1883,54565,0.000246,8,632629,1.0,8.0,...,0,0,0,0,0,0,0,0,1.0,1
37181,0.000028,20.368755,398,54542,1883,0.000028,344,36061,99.0,344.0,...,0,0,0,0,0,0,0,0,1.0,1
85581,0.000030,500.819858,84,38619,1883,4.997798,16,21,6.0,16.0,...,0,0,0,0,0,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154039,0.000093,2779.034682,77,41889,1883,2.006452,9,97,4.0,9.0,...,0,0,0,0,0,0,0,0,0.0,0
138330,0.000043,1059.031537,77,35965,1883,2.000624,9,39,4.0,9.0,...,0,0,0,0,0,0,0,0,0.0,0
171226,0.000071,4673.029847,77,38323,1883,2.000755,9,159,4.0,9.0,...,0,0,0,0,0,0,0,0,0.0,0
37187,0.000027,20.371876,54,54542,1883,0.000027,0,36097,1.0,1.0,...,0,0,0,0,0,0,0,0,1.0,1


In [85]:
pred = basic_model.predict(x_test_scaled)
X_test['pred'] = np.argmax(pred, axis=1)

In [90]:
(X_test['label_true'] == X_test['pred']).value_counts()

True     28273
False       32
dtype: int64

In [91]:
X_test

,frame.time_delta,frame.time_relative,frame.len,tcp.srcport,tcp.dstport,tcp.time_delta,tcp.len,tcp.ack,mqtt.msg,payload_len,...,src_ip192.168.1.91,src_ip10.5.126.145,src_ip10.5.126.147,src_ip10.5.126.148,src_ip10.5.126.146,dst_ip 10.5.126.56,dst_ip 192.168.1.91,dst_ip 192.168.1.90,label_true,pred
132052,0.000042,367.029651,86,33571,1883,1.999646,18,17,4.0,18.0,...,0,0,0,0,0,0,0,0,0.0,0
147977,0.000112,2113.041946,86,37701,1883,2.009467,18,75,4.0,18.0,...,0,0,0,0,0,0,0,0,0.0,0
58671,0.000165,53.493371,62,1883,54565,0.000246,8,632629,1.0,8.0,...,0,0,0,0,0,0,0,0,1.0,1
37181,0.000028,20.368755,398,54542,1883,0.000028,344,36061,99.0,344.0,...,0,0,0,0,0,0,0,0,1.0,1
85581,0.000030,500.819858,84,38619,1883,4.997798,16,21,6.0,16.0,...,0,0,0,0,0,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154039,0.000093,2779.034682,77,41889,1883,2.006452,9,97,4.0,9.0,...,0,0,0,0,0,0,0,0,0.0,0
138330,0.000043,1059.031537,77,35965,1883,2.000624,9,39,4.0,9.0,...,0,0,0,0,0,0,0,0,0.0,0
171226,0.000071,4673.029847,77,38323,1883,2.000755,9,159,4.0,9.0,...,0,0,0,0,0,0,0,0,0.0,0
37187,0.000027,20.371876,54,54542,1883,0.000027,0,36097,1.0,1.0,...,0,0,0,0,0,0,0,0,1.0,1
